# SOM implementation in Python using Sklearn/SciKit

### Imports

In [33]:
# Building SOM
from sklearn import datasets, metrics # Contains IRIS dataset
from sklearn_som.som import SOM
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import subprocess as sp
import os

# Plotting graph
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Measuring time
import time


### Data loading

In [ ]:
# IRIS data loading
iris = datasets.load_iris()
iris_data = iris.data[:, :2]
iris_label = iris.target

# BME data loading
bme_file = open("datasets/BME_MOD.txt", "r")
bme_data = []
for line in bme_file:
    bme_data.append(line.split())
    for i in range(0, 129):
        bme_data[-1][i] = float(bme_data[-1][i])


# Confirm all 180 time series (train + test) loaded into array
len(bme_data)

# Convert python array to numpy array
bme_data = np.array(bme_data)

# Scaling data
scaler = MinMaxScaler()
bme_data = scaler.fit_transform(bme_data)

### Building SOMs

In [32]:

# SOM for IRIS dataset
iris_som = SOM(m = 3, n = 1, dim = 2)   # 3x1 structure for the SOM

st_iris_fit = time.time()
iris_som.fit(iris_data)
et_iris_fit = time.time()

# SOM for BME dataset
bme_som = SOM(m = 3, n = 1, dim = 129)

st_bme_fit = time.time()
bme_som.fit(bme_data)
et_bme_fit = time.time()

### Applying datasets to SOMs

In [ ]:
st_iris_pred = time.time()
iris_predictions = iris_som.predict(iris_data)
et_iris_pred = time.time()

st_bme_pred = time.time()
bme_predictions = bme_som.predict(bme_data)
et_bme_pred = time.time()


### Plotting graphs

In [ ]:
# Plot the results
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,12))
x_iris = iris_data[:,0]
y_iris = iris_data[:,1]
x_bme = bme_data[:, 0]
y_bme = bme_data[:, 1]
colors = ['red', 'green', 'blue']

ax[0,0].scatter(x_iris, y_iris, c=iris_label, cmap=ListedColormap(colors))
ax[0,0].title.set_text('Actual Classes for IRIS dataset')
ax[0,1].scatter(x_iris, y_iris, c=iris_predictions, cmap=ListedColormap(colors))
ax[0,1].title.set_text('SOM Predictions for IRIS dataset')
ax[1,0].scatter(x_bme, y_bme, c=bme_data[:, 0])
ax[1,0].title.set_text('Actual Classes for BME dataset')
ax[1,1].scatter(x_bme, y_bme, c=bme_predictions)
ax[1,1].title.set_text('SOM Predictions for BME dataset')

### RAND Score

In [ ]:
print("RAND Scores:\n IRIS - Sklearn:", metrics.rand_score(iris_label, iris_predictions), "\nBME - Sklearn: ", metrics.rand_score(bme_data[0, :], bme_predictions))

### Retrieve c-code's output

In [ ]:
# cmd = '''
#     gcc -o som som.c -lm && ./som bme
# '''
# print(os.getcwd())
# print(os.listdir('.'))
# output = sp.check_output(cmd.strip('\n').split(), shell=True)
# print(output.decode('utf-8'))